In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from pydataset import data

#import warnings
#warnings.filterwarnings("ignore")

## Load the tips dataset.

* Create a column named price_per_person. This should be the total bill divided by the party size.
* Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
* Use select k best to select the top 2 features for predicting tip amount. What are they?
* Use recursive feature elimination to select the top 2 features for tip amount. What are they?
* Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?'''


In [2]:
df = data('tips')

In [3]:
df.insert(7, "price_per_person",(df.total_bill/df.size), True)

In [4]:
df.corr()

,total_bill,tip,size,price_per_person
total_bill,1.000000,0.675734,0.598315,1.000000
tip,0.675734,1.000000,0.489299,0.675734
size,0.598315,0.489299,1.000000,0.598315
price_per_person,1.000000,0.675734,0.598315,1.000000


In [5]:
def train_val_test(df, target=None, stratify=None, seed=42):
    from sklearn.model_selection import train_test_split

    '''Split data into train, validate, and test subsets with 60/20/20 ratio'''
    
    train, val_test = train_test_split(df, train_size=0.6, random_state=seed)
    
    val, test = train_test_split(val_test, train_size=0.5, random_state=seed)
    
    return train, val, test

In [6]:
train, val, test = train_val_test(df,stratify='tip')

In [7]:
categorical_variables = ['sex','smoker','day','time']
continous_variables = ['total_bill','size','price_per_person']
target = ['tip']


In [8]:
X_train_scaled = pd.DataFrame(train[continous_variables])
y_train = train[target]

In [9]:
mms = MinMaxScaler()

In [10]:
X_train_scaled[continous_variables] = mms.fit_transform(X_train_scaled)
X_train_scaled.head()

,total_bill,size,price_per_person
195,0.240346,0.2,0.240346
77,0.270084,0.2,0.270084
42,0.259876,0.2,0.259876
109,0.277186,0.2,0.277186
224,0.227031,0.4,0.227031


In [11]:
f_selector = SelectKBest(f_regression, k=2)

f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
f_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [13]:
f_feature

['total_bill', 'price_per_person']

In [14]:
X_train_scaled.head()

,total_bill,size,price_per_person
195,0.240346,0.2,0.240346
77,0.270084,0.2,0.270084
42,0.259876,0.2,0.259876
109,0.277186,0.2,0.277186
224,0.227031,0.4,0.227031


In [15]:
X_train_scaled = pd.concat([X_train_scaled, train[categorical_variables]], axis=1)

In [16]:
X_train_scaled.head()

,total_bill,size,price_per_person,sex,smoker,day,time
195,0.240346,0.2,0.240346,Male,Yes,Thur,Lunch
77,0.270084,0.2,0.270084,Male,Yes,Sat,Dinner
42,0.259876,0.2,0.259876,Male,No,Sun,Dinner
109,0.277186,0.2,0.277186,Male,No,Sat,Dinner
224,0.227031,0.4,0.227031,Female,No,Fri,Lunch


In [17]:
X_train_scaled = pd.get_dummies(X_train_scaled, columns = categorical_variables)

In [18]:
X_train_scaled.head()


,total_bill,size,price_per_person,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
195,0.240346,0.2,0.240346,0,1,0,1,0,0,0,1,0,1
77,0.270084,0.2,0.270084,0,1,0,1,0,1,0,0,1,0
42,0.259876,0.2,0.259876,0,1,1,0,0,0,1,0,1,0
109,0.277186,0.2,0.277186,0,1,1,0,0,1,0,0,1,0
224,0.227031,0.4,0.227031,1,0,1,0,1,0,0,0,0,1


In [19]:
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, n_features_to_select=2)

# fit the data using RFE
rfe.fit(X_train_scaled, y_train)

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train_scaled.iloc[:,feature_mask].columns.tolist()
rfe_feature

['total_bill', 'smoker_Yes']

In [20]:
var_ranks = rfe.ranking_

# get the variable names
var_names = X_train_scaled.columns.tolist()

# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})

# sort the df by rank
rfe_ranks_df.sort_values('Rank').head(2)

,Var,Rank
0,total_bill,1
6,smoker_Yes,1


In [21]:
rfe_ranks_df.sort_values('Rank').head(2)

,Var,Rank
0,total_bill,1
6,smoker_Yes,1


In [ ]:
# Change number of Features

## 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
def select_kbest(X, y, k=2):
    f_selector = SelectKBest(f_regression, k = k)
    f_selector.fit(X, y)
    mask = f_selector.get_support()
    
    return X.columns[mask].tolist()

In [23]:
X = pd.DataFrame(train[continous_variables])
y = train[target]

select_kbest(X,y)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['total_bill', 'price_per_person']

## 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [26]:
def rfe(X, y, k=2):
    rfe = RFE(LinearRegression(), n_features_to_select = k)
    rfe.fit(X, y)
    feature_mask = rfe.support_
    
    return X.iloc[:,feature_mask].columns.tolist()

In [27]:
X = pd.DataFrame(train[continous_variables])
X = pd.concat([X, train[categorical_variables]], axis=1)
X = pd.get_dummies(X, columns = categorical_variables)

rfe(X,y)

['size', 'sex_Female']

## 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).